In [1]:
import time

import pandas as pd
import numpy as np
import json
import sqlite3

In [27]:
# Đọc data từ db
with sqlite3.connect('data/address-converter.io.vn_convert.db') as conn:
    conn.row_factory = sqlite3.Row
    cursor = conn.cursor()
    result = cursor.execute('''SELECT * FROM convert''')
    records = [dict(row) for row in result.fetchall()]

# Lọc data không có lỗi
converts = [json.loads(i['data']) for i in records]
df_convert = pd.DataFrame(converts)
df_convert = df_convert[df_convert['error'].isna()]

# Bung cột ra
df_expanded = pd.concat([
    df_convert.drop(['oldAddress', 'newAddress'], axis=1),
    df_convert['oldAddress'].apply(pd.Series).add_prefix('old_'),
    df_convert['newAddress'].apply(pd.Series).add_prefix('new_')
], axis=1)
df_expanded.drop_duplicates(inplace=True)

In [28]:
# Đọc data ward
df_ward = pd.read_csv('data/address-converter.io.vn_ward.csv', dtype={'wardCode': str, 'districtCode': str, 'provinceCode': str})
# df_ward.fillna('', inplace=True)

In [29]:
# Kiểm tra còn ward nào chưa scrap
df_ward[(~df_ward['wardCode'].isin(df_expanded['old_wardCode'])) & (~df_ward['wardCode'].isna())]

,wardName,wardCode,districtName,districtCode,provinceName,provinceCode


In [30]:
# Xóa các cột không có data
df_expanded.drop(columns=['success', 'error', 'old_detail', 'new_detail', 'new_district', 'new_districtCode'], inplace=True)

In [31]:
# Đổi tên để chuẩn bị cho merge
df_ward.rename(columns=lambda x: f'old_{x}', inplace=True)

In [46]:
# Merge data
df_merge = pd.merge(df_ward, df_expanded, on=['old_wardCode', 'old_districtCode', 'old_provinceCode'], how='left')

In [47]:
# Kiểm tra có bao nhiều phường không chắc chắn chính xác
df_merge[df_merge['notSure']==True].shape[0]

89

In [56]:
# Kiểm tra các đảo để cập nhật thủ công vì API của họ bắt buộc có wardCode mà đảo thì đâu có wardCode
df_merge[df_merge['old_wardCode'].isna()]

,old_wardCode,old_districtCode,old_provinceCode,old_ward,old_district,old_province,old_fullAddress,new_wardCode,new_provinceCode,new_ward,new_province,new_fullAddress,notSure,note
230,NaN,318,31,NaN,Huyện Bạch Long Vĩ,Thành phố Hải Phòng,"Huyện Bạch Long Vĩ, Thành phố Hải Phòng",NaN,31,Đặc khu Bạch Long Vĩ,Thành phố Hải Phòng,"Đặc khu Bạch Long Vĩ, Thành phố Hải Phòng",False,NaN
654,NaN,498,48,NaN,Huyện Hoàng Sa,Thành phố Đà Nẵng,"Huyện Hoàng Sa, Thành phố Đà Nẵng",NaN,48,Đặc khu Hoàng Sa,Thành phố Đà Nẵng,"Đặc khu Hoàng Sa, Thành phố Đà Nẵng",False,NaN
873,NaN,755,77,NaN,Huyện Côn Đảo,Tỉnh Bà Rịa - Vũng Tàu,"Huyện Côn Đảo, Tỉnh Bà Rịa - Vũng Tàu",NaN,79,Đặc khu Côn Đảo,Thành phố Hồ Chí Minh,"Đặc khu Côn Đảo, Thành phố Hồ Chí Minh",False,NaN
5639,NaN,536,51,NaN,Huyện Lý Sơn,Tỉnh Quảng Ngãi,"Huyện Lý Sơn, Tỉnh Quảng Ngãi",NaN,51,Đặc khu Lý Sơn,Quảng Ngãi,"Đặc khu Lý Sơn, Quảng Ngãi",False,NaN
5948,NaN,471,45,NaN,Huyện Cồn Cỏ,Tỉnh Quảng Trị,"Huyện Cồn Cỏ, Tỉnh Quảng Trị",NaN,44,Đặc khu Cồn Cỏ,Quảng Trị,"Đặc khu Cồn Cỏ, Quảng Trị",False,NaN


In [49]:
# Cập nhật bằng tay, tham khảo thêm https://vnexpress.net/tra-cuu-3-321-phuong-xa-tren-ca-nuoc-sau-sap-xep-4903454.html
df_merge.loc[df_merge['old_districtName']=='Huyện Bạch Long Vĩ', 'notSure'] = False
df_merge.loc[df_merge['old_districtName']=='Huyện Bạch Long Vĩ', 'old_fullAddress'] = 'Huyện Bạch Long Vĩ, Thành phố Hải Phòng'
df_merge.loc[df_merge['old_districtName']=='Huyện Bạch Long Vĩ', 'new_ward'] = 'Đặc khu Bạch Long Vĩ'
df_merge.loc[df_merge['old_districtName']=='Huyện Bạch Long Vĩ', 'new_province'] = 'Thành phố Hải Phòng'
df_merge.loc[df_merge['old_districtName']=='Huyện Bạch Long Vĩ', 'new_provinceCode'] = '31'
df_merge.loc[df_merge['old_districtName']=='Huyện Bạch Long Vĩ', 'new_fullAddress'] = 'Đặc khu Bạch Long Vĩ, Thành phố Hải Phòng'

df_merge.loc[df_merge['old_districtName']=='Huyện Hoàng Sa', 'notSure'] = False
df_merge.loc[df_merge['old_districtName']=='Huyện Hoàng Sa', 'old_fullAddress'] = 'Huyện Hoàng Sa, Thành phố Đà Nẵng'
df_merge.loc[df_merge['old_districtName']=='Huyện Hoàng Sa', 'new_ward'] = 'Đặc khu Hoàng Sa'
df_merge.loc[df_merge['old_districtName']=='Huyện Hoàng Sa', 'new_province'] = 'Thành phố Đà Nẵng'
df_merge.loc[df_merge['old_districtName']=='Huyện Hoàng Sa', 'new_provinceCode'] = '48'
df_merge.loc[df_merge['old_districtName']=='Huyện Hoàng Sa', 'new_fullAddress'] = 'Đặc khu Hoàng Sa, Thành phố Đà Nẵng'

df_merge.loc[df_merge['old_districtName']=='Huyện Lý Sơn', 'notSure'] = False
df_merge.loc[df_merge['old_districtName']=='Huyện Lý Sơn', 'old_fullAddress'] = 'Huyện Lý Sơn, Tỉnh Quảng Ngãi'
df_merge.loc[df_merge['old_districtName']=='Huyện Lý Sơn', 'new_ward'] = 'Đặc khu Lý Sơn'
df_merge.loc[df_merge['old_districtName']=='Huyện Lý Sơn', 'new_province'] = 'Quảng Ngãi'
df_merge.loc[df_merge['old_districtName']=='Huyện Lý Sơn', 'new_provinceCode'] = '51'
df_merge.loc[df_merge['old_districtName']=='Huyện Lý Sơn', 'new_fullAddress'] = 'Đặc khu Lý Sơn, Quảng Ngãi'

df_merge.loc[df_merge['old_districtName']=='Huyện Cồn Cỏ', 'notSure'] = False
df_merge.loc[df_merge['old_districtName']=='Huyện Cồn Cỏ', 'old_fullAddress'] = 'Huyện Cồn Cỏ, Tỉnh Quảng Trị'
df_merge.loc[df_merge['old_districtName']=='Huyện Cồn Cỏ', 'new_ward'] = 'Đặc khu Cồn Cỏ'
df_merge.loc[df_merge['old_districtName']=='Huyện Cồn Cỏ', 'new_province'] = 'Quảng Trị'
df_merge.loc[df_merge['old_districtName']=='Huyện Cồn Cỏ', 'new_provinceCode'] = '44'
df_merge.loc[df_merge['old_districtName']=='Huyện Cồn Cỏ', 'new_fullAddress'] = 'Đặc khu Cồn Cỏ, Quảng Trị'

df_merge.loc[df_merge['old_districtName']=='Huyện Côn Đảo', 'notSure'] = False
df_merge.loc[df_merge['old_districtName']=='Huyện Côn Đảo', 'old_fullAddress'] = 'Huyện Côn Đảo, Tỉnh Bà Rịa - Vũng Tàu'
df_merge.loc[df_merge['old_districtName']=='Huyện Côn Đảo', 'new_ward'] = 'Đặc khu Côn Đảo'
df_merge.loc[df_merge['old_districtName']=='Huyện Côn Đảo', 'new_province'] = 'Thành phố Hồ Chí Minh'
df_merge.loc[df_merge['old_districtName']=='Huyện Côn Đảo', 'new_provinceCode'] = '79'
df_merge.loc[df_merge['old_districtName']=='Huyện Côn Đảo', 'new_fullAddress'] = 'Đặc khu Côn Đảo, Thành phố Hồ Chí Minh'

In [51]:
# Bỏ bớt cột trùng lặp, ưu tiên của df_ward vì nó là gốc
df_merge.drop(columns=['old_district', 'old_province', 'old_ward'], inplace=True)
df_merge.rename(columns={'old_wardName': 'old_ward', 'old_districtName': 'old_district', 'old_provinceName': 'old_province'}, inplace=True)

In [53]:
# Sắp xếp lại thứ tự cột cho dễ xem
sorted_cols = [
 'old_wardCode',
 'old_districtCode',
 'old_provinceCode',

 'old_ward',
 'old_district',
 'old_province',
 'old_fullAddress',

 'new_wardCode',
 'new_provinceCode',
 'new_ward',
 'new_province',
 'new_fullAddress',

 'notSure',
 'note'
]

df_merge = df_merge[sorted_cols].copy()

In [57]:
df_merge.to_csv('data/address-converter.io.vn_convert.csv', index=False)

In [58]:
df_merge

,old_wardCode,old_districtCode,old_provinceCode,old_ward,old_district,old_province,old_fullAddress,new_wardCode,new_provinceCode,new_ward,new_province,new_fullAddress,notSure,note
0,31261,925,92,Thị trấn Cờ Đỏ,Huyện Cờ Đỏ,Thành phố Cần Thơ,"Thị trấn Cờ Đỏ, Huyện Cờ Đỏ, Thành phố Cần Thơ",31261,92,Xã Cờ Đỏ,Thành phố Cần Thơ,"Xã Cờ Đỏ, Thành phố Cần Thơ",False,None
1,31249,925,92,Xã Thạnh Phú,Huyện Cờ Đỏ,Thành phố Cần Thơ,"Xã Thạnh Phú, Huyện Cờ Đỏ, Thành phố Cần Thơ",31249,92,Xã Thạnh Phú,Thành phố Cần Thơ,"Xã Thạnh Phú, Thành phố Cần Thơ",False,None
2,31264,925,92,Xã Thới Hưng,Huyện Cờ Đỏ,Thành phố Cần Thơ,"Xã Thới Hưng, Huyện Cờ Đỏ, Thành phố Cần Thơ",31264,92,Xã Thới Hưng,Thành phố Cần Thơ,"Xã Thới Hưng, Thành phố Cần Thơ",False,None
3,31277,925,92,Xã Thới Xuân,Huyện Cờ Đỏ,Thành phố Cần Thơ,"Xã Thới Xuân, Huyện Cờ Đỏ, Thành phố Cần Thơ",31261,92,Xã Cờ Đỏ,Thành phố Cần Thơ,"Xã Cờ Đỏ, Thành phố Cần Thơ",False,None
4,31276,925,92,Xã Thới Đông,Huyện Cờ Đỏ,Thành phố Cần Thơ,"Xã Thới Đông, Huyện Cờ Đỏ, Thành phố Cần Thơ",31261,92,Xã Cờ Đỏ,Thành phố Cần Thơ,"Xã Cờ Đỏ, Thành phố Cần Thơ",False,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8687,29919,867,87,Phường An Hoà,Thành phố Sa Đéc,Tỉnh Đồng Tháp,"Phường An Hoà, Thành phố Sa Đéc, Tỉnh Đồng Tháp",29905,82,Phường Sa Đéc,Đồng Tháp,"Phường Sa Đéc, Đồng Tháp",False,None
8688,29917,867,87,Phường Tân Quy Đông,Thành phố Sa Đéc,Tỉnh Đồng Tháp,"Phường Tân Quy Đông, Thành phố Sa Đéc, Tỉnh Đồ...",29905,82,Phường Sa Đéc,Đồng Tháp,"Phường Sa Đéc, Đồng Tháp",False,None
8689,29914,867,87,Xã Tân Khánh Đông,Thành phố Sa Đéc,Tỉnh Đồng Tháp,"Xã Tân Khánh Đông, Thành phố Sa Đéc, Tỉnh Đồng...",29905,82,Phường Sa Đéc,Đồng Tháp,"Phường Sa Đéc, Đồng Tháp",False,None
8690,29923,867,87,Xã Tân Phú Đông,Thành phố Sa Đéc,Tỉnh Đồng Tháp,"Xã Tân Phú Đông, Thành phố Sa Đéc, Tỉnh Đồng Tháp",30214,82,Xã Tân Dương,Đồng Tháp,"Xã Tân Dương, Đồng Tháp",False,None


In [100]:
not_sure_district = df_merge[df_merge['notSure']==True][['old_province', 'old_district']].drop_duplicates()

In [64]:
# not_sure_district.to_csv('data/address-converter.io.vn_not_sure_district', index=False)

In [102]:
not_sure_district.loc[(not_sure_district['old_province']=='Thành phố Hồ Chí Minh') & (not_sure_district['old_district']=='Quận Tân Phú'), 'wikipedia'] = 'https://vi.wikipedia.org/wiki/T%C3%A2n_Ph%C3%BA_(qu%E1%BA%ADn)#%C4%90%C6%B0%E1%BB%9Dng_ph%E1%BB%91'
not_sure_district.loc[(not_sure_district['old_province']=='Thành phố Hồ Chí Minh') & (not_sure_district['old_district']=='Huyện Bình Chánh'), 'wikipedia'] = 'https://vi.wikipedia.org/wiki/B%C3%ACnh_Ch%C3%A1nh_(huy%E1%BB%87n)#%C4%90%C6%B0%E1%BB%9Dng_ph%E1%BB%91'
not_sure_district.loc[(not_sure_district['old_province']=='Thành phố Hồ Chí Minh') & (not_sure_district['old_district']=='Quận 1'), 'wikipedia'] = 'https://vi.wikipedia.org/wiki/Qu%E1%BA%ADn_1#%C4%90%C6%B0%E1%BB%9Dng_ph%E1%BB%91'
not_sure_district.loc[(not_sure_district['old_province']=='Thành phố Hồ Chí Minh') & (not_sure_district['old_district']=='Quận 10'), 'wikipedia'] = 'https://vi.wikipedia.org/wiki/Qu%E1%BA%ADn_10#%C4%90%C6%B0%E1%BB%9Dng_ph%E1%BB%91'
not_sure_district.loc[(not_sure_district['old_province']=='Thành phố Hồ Chí Minh') & (not_sure_district['old_district']=='Quận 11'), 'wikipedia'] = 'https://vi.wikipedia.org/wiki/Qu%E1%BA%ADn_11#%C4%90%C6%B0%E1%BB%9Dng_ph%E1%BB%91'
not_sure_district.loc[(not_sure_district['old_province']=='Thành phố Hồ Chí Minh') & (not_sure_district['old_district']=='Quận 3'), 'wikipedia'] = 'https://vi.wikipedia.org/wiki/Qu%E1%BA%ADn_3#%C4%90%C6%B0%E1%BB%9Dng_ph%E1%BB%91'
not_sure_district.loc[(not_sure_district['old_province']=='Thành phố Hồ Chí Minh') & (not_sure_district['old_district']=='Quận 4'), 'wikipedia'] = 'https://vi.wikipedia.org/wiki/Qu%E1%BA%ADn_4#%C4%90%C6%B0%E1%BB%9Dng_ph%E1%BB%91'
not_sure_district.loc[(not_sure_district['old_province']=='Thành phố Hồ Chí Minh') & (not_sure_district['old_district']=='Quận 7'), 'wikipedia'] = 'https://vi.wikipedia.org/wiki/Qu%E1%BA%ADn_7#%C4%90%C6%B0%E1%BB%9Dng_ph%E1%BB%91'
not_sure_district.loc[(not_sure_district['old_province']=='Thành phố Hồ Chí Minh') & (not_sure_district['old_district']=='Quận 8'), 'wikipedia'] = 'https://vi.wikipedia.org/wiki/Qu%E1%BA%ADn_8#%C4%90%C6%B0%E1%BB%9Dng_ph%E1%BB%91'
not_sure_district.loc[(not_sure_district['old_province']=='Thành phố Hồ Chí Minh') & (not_sure_district['old_district']=='Quận Bình Tân'), 'wikipedia'] = 'https://vi.wikipedia.org/wiki/B%C3%ACnh_T%C3%A2n_(qu%E1%BA%ADn)#%C4%90%C6%B0%E1%BB%9Dng_ph%E1%BB%91'
not_sure_district.loc[(not_sure_district['old_province']=='Thành phố Hồ Chí Minh') & (not_sure_district['old_district']=='Quận Phú Nhuận'), 'wikipedia'] = 'https://vi.wikipedia.org/wiki/Ph%C3%BA_Nhu%E1%BA%ADn_(qu%E1%BA%ADn)#%C4%90%C6%B0%E1%BB%9Dng_ph%E1%BB%91'
not_sure_district.loc[(not_sure_district['old_province']=='Thành phố Hồ Chí Minh') & (not_sure_district['old_district']=='Quận Tân Bình'), 'wikipedia'] = 'https://vi.wikipedia.org/wiki/T%C3%A2n_B%C3%ACnh_(qu%E1%BA%ADn)#%C4%90%C6%B0%E1%BB%9Dng_ph%E1%BB%91'
not_sure_district.loc[(not_sure_district['old_province']=='Thành phố Hồ Chí Minh') & (not_sure_district['old_district']=='Thành phố Thủ Đức'), 'wikipedia'] = 'https://vi.wikipedia.org/wiki/Th%E1%BB%A7_%C4%90%E1%BB%A9c_(th%C3%A0nh_ph%E1%BB%91)#%C4%90%C6%B0%E1%BB%9Dng_ph%E1%BB%91'
not_sure_district.loc[(not_sure_district['old_province']=='Thành phố Cần Thơ') & (not_sure_district['old_district']=='Quận Bình Thuỷ'), 'wikipedia'] = 'https://vi.wikipedia.org/wiki/B%C3%ACnh_Th%E1%BB%A7y_(qu%E1%BA%ADn)#%C4%90%C6%B0%E1%BB%9Dng_ph%E1%BB%91'
not_sure_district.loc[(not_sure_district['old_province']=='Thành phố Cần Thơ') & (not_sure_district['old_district']=='Quận Thốt Nốt'), 'wikipedia'] = 'https://vi.wikipedia.org/wiki/Th%E1%BB%91t_N%E1%BB%91t_(qu%E1%BA%ADn)#%C4%90%C6%B0%E1%BB%9Dng_ph%E1%BB%91'
not_sure_district.loc[(not_sure_district['old_province']=='Thành phố Hải Phòng') & (not_sure_district['old_district']=='Huyện An Lão'), 'wikipedia'] = 'NO'
not_sure_district.loc[(not_sure_district['old_province']=='Thành phố Hải Phòng') & (not_sure_district['old_district']=='Quận An Dương'), 'wikipedia'] = 'https://vi.wikipedia.org/wiki/An_D%C6%B0%C6%A1ng_(qu%E1%BA%ADn)#%C4%90%C6%B0%E1%BB%9Dng_ph%E1%BB%91'
not_sure_district.loc[(not_sure_district['old_province']=='Thành phố Hải Phòng') & (not_sure_district['old_district']=='Quận Dương Kinh'), 'wikipedia'] = 'https://vi.wikipedia.org/wiki/D%C6%B0%C6%A1ng_Kinh_(qu%E1%BA%ADn)#%C4%90%C6%B0%E1%BB%9Dng_ph%E1%BB%91'
not_sure_district.loc[(not_sure_district['old_province']=='Thành phố Hải Phòng') & (not_sure_district['old_district']=='Quận Hải An'), 'wikipedia'] = 'https://vi.wikipedia.org/wiki/H%E1%BA%A3i_An_(qu%E1%BA%ADn)#%C4%90%C6%B0%E1%BB%9Dng_ph%E1%BB%91'
not_sure_district.loc[(not_sure_district['old_province']=='Thành phố Hải Phòng') & (not_sure_district['old_district']=='Quận Hồng Bàng'), 'wikipedia'] = 'https://vi.wikipedia.org/wiki/H%E1%BB%93ng_B%C3%A0ng_(qu%E1%BA%ADn)#%C4%90%C6%B0%E1%BB%9Dng_ph%E1%BB%91'
not_sure_district.loc[(not_sure_district['old_province']=='Thành phố Hải Phòng') & (not_sure_district['old_district']=='Quận Kiến An'), 'wikipedia'] = 'https://vi.wikipedia.org/wiki/Ki%E1%BA%BFn_An_(qu%E1%BA%ADn)#%C4%90%C6%B0%E1%BB%9Dng_ph%E1%BB%91'
not_sure_district.loc[(not_sure_district['old_province']=='Thành phố Hải Phòng') & (not_sure_district['old_district']=='Quận Lê Chân'), 'wikipedia'] = 'https://vi.wikipedia.org/wiki/L%C3%AA_Ch%C3%A2n_(qu%E1%BA%ADn)#Giao_th%C3%B4ng'
not_sure_district.loc[(not_sure_district['old_province']=='Thành phố Hải Phòng') & (not_sure_district['old_district']=='Quận Ngô Quyền'), 'wikipedia'] = 'https://vi.wikipedia.org/wiki/Ng%C3%B4_Quy%E1%BB%81n_(qu%E1%BA%ADn)#%C4%90%C6%B0%E1%BB%9Dng_ph%E1%BB%91'
not_sure_district.loc[(not_sure_district['old_province']=='Thành phố Hải Phòng') & (not_sure_district['old_district']=='Quận Đồ Sơn'), 'wikipedia'] = 'https://vi.wikipedia.org/wiki/%C4%90%E1%BB%93_S%C6%A1n_(qu%E1%BA%ADn)#%C4%90%C6%B0%E1%BB%9Dng_ph%E1%BB%91'
not_sure_district.loc[(not_sure_district['old_province']=='Thành phố Hải Phòng') & (not_sure_district['old_district']=='Thành phố Thuỷ Nguyên'), 'wikipedia'] = 'https://vi.wikipedia.org/wiki/Th%E1%BB%A7y_Nguy%C3%AAn_(th%C3%A0nh_ph%E1%BB%91)#Giao_th%C3%B4ng'
not_sure_district.loc[(not_sure_district['old_province']=='Tỉnh Cà Mau') & (not_sure_district['old_district']=='Thành phố Cà Mau'), 'wikipedia'] = 'https://vi.wikipedia.org/wiki/C%C3%A0_Mau_(th%C3%A0nh_ph%E1%BB%91)#Giao_th%C3%B4ng'
not_sure_district.loc[(not_sure_district['old_province']=='Tỉnh Hà Nam') & (not_sure_district['old_district']=='Thành phố Phủ Lý'), 'wikipedia'] = 'https://vi.wikipedia.org/wiki/Ph%E1%BB%A7_L%C3%BD#%C4%90%C6%B0%E1%BB%9Dng_ph%E1%BB%91'
not_sure_district.loc[(not_sure_district['old_province']=='Tỉnh Hà Nam') & (not_sure_district['old_district']=='Thị xã Duy Tiên'), 'wikipedia'] = 'https://vi.wikipedia.org/wiki/Duy_Ti%C3%AAn#%C4%90%C6%B0%E1%BB%9Dng_ph%E1%BB%91'
not_sure_district.loc[(not_sure_district['old_province']=='Tỉnh Hà Tĩnh') & (not_sure_district['old_district']=='Thành phố Hà Tĩnh'), 'wikipedia'] = 'NO'
not_sure_district.loc[(not_sure_district['old_province']=='Tỉnh Hải Dương') & (not_sure_district['old_district']=='Huyện Cẩm Giàng'), 'wikipedia'] = 'NO'
not_sure_district.loc[(not_sure_district['old_province']=='Tỉnh Hải Dương') & (not_sure_district['old_district']=='Huyện Gia Lộc'), 'wikipedia'] = 'NO'
not_sure_district.loc[(not_sure_district['old_province']=='Tỉnh Hải Dương') & (not_sure_district['old_district']=='Huyện Thanh Miện'), 'wikipedia'] = 'NO'
not_sure_district.loc[(not_sure_district['old_province']=='Tỉnh Hải Dương') & (not_sure_district['old_district']=='Huyện Tứ Kỳ'), 'wikipedia'] = 'NO'
not_sure_district.loc[(not_sure_district['old_province']=='Tỉnh Hải Dương') & (not_sure_district['old_district']=='Thành phố Chí Linh'), 'wikipedia'] = 'NO'
not_sure_district.loc[(not_sure_district['old_province']=='Tỉnh Hải Dương') & (not_sure_district['old_district']=='Thành phố Hải Dương'), 'wikipedia'] = 'NO'
not_sure_district.loc[(not_sure_district['old_province']=='Tỉnh Phú Yên') & (not_sure_district['old_district']=='Huyện Phú Hoà'), 'wikipedia'] = 'NO'
not_sure_district.loc[(not_sure_district['old_province']=='Tỉnh Phú Yên') & (not_sure_district['old_district']=='Huyện Tây Hoà'), 'wikipedia'] = 'NO'
not_sure_district.loc[(not_sure_district['old_province']=='Tỉnh Phú Yên') & (not_sure_district['old_district']=='Thị xã Đông Hòa'), 'wikipedia'] = 'NO'
not_sure_district.loc[(not_sure_district['old_province']=='Tỉnh Quảng Ninh') & (not_sure_district['old_district']=='Thành phố Hạ Long'), 'wikipedia'] = 'https://vi.wikipedia.org/wiki/H%E1%BA%A1_Long#%C4%90%C6%B0%E1%BB%9Dng_ph%E1%BB%91'
not_sure_district.loc[(not_sure_district['old_province']=='Tỉnh Quảng Ninh') & (not_sure_district['old_district']=='Thành phố Uông Bí'), 'wikipedia'] = 'https://vi.wikipedia.org/wiki/U%C3%B4ng_B%C3%AD#C%C3%A1c_%C4%91%E1%BB%8Ba_danh,_%C4%91%C6%B0%E1%BB%9Dng_ph%E1%BB%91,_c%C3%B4ng_tr%C3%ACnh_c%C3%B4ng_c%E1%BB%99ng_t%E1%BA%A1i_U%C3%B4ng_B%C3%AD'
not_sure_district.loc[(not_sure_district['old_province']=='Tỉnh Quảng Ninh') & (not_sure_district['old_district']=='Thành phố Đông Triều'), 'wikipedia'] = 'https://vi.wikipedia.org/wiki/%C4%90%C3%B4ng_Tri%E1%BB%81u#%C4%90%C6%B0%E1%BB%9Dng_ph%E1%BB%91'
not_sure_district.loc[(not_sure_district['old_province']=='Tỉnh Thanh Hóa') & (not_sure_district['old_district']=='Thành phố Thanh Hóa'), 'wikipedia'] = 'https://vi.wikipedia.org/wiki/Thanh_H%C3%B3a_(th%C3%A0nh_ph%E1%BB%91)#Danh_s%C3%A1ch_%C4%91%C6%B0%E1%BB%9Dng_ph%E1%BB%91'
not_sure_district.loc[(not_sure_district['old_province']=='Tỉnh Thái Nguyên') & (not_sure_district['old_district']=='Thành phố Thái Nguyên'), 'wikipedia'] = 'https://vi.wikipedia.org/wiki/Th%C3%A1i_Nguy%C3%AAn_(th%C3%A0nh_ph%E1%BB%91)#Giao_th%C3%B4ng'

In [146]:
if not_sure_district[not_sure_district['wikipedia'].isna()].shape[0]:
    raise Exception('Thiếu wikipedia')

In [104]:
not_sure_district

,old_province,old_district,wikipedia
43,Thành phố Cần Thơ,Quận Bình Thuỷ,https://vi.wikipedia.org/wiki/B%C3%ACnh_Th%E1%...
67,Thành phố Cần Thơ,Quận Thốt Nốt,https://vi.wikipedia.org/wiki/Th%E1%BB%91t_N%E...
214,Thành phố Hải Phòng,Huyện An Lão,NO
299,Thành phố Hải Phòng,Quận An Dương,https://vi.wikipedia.org/wiki/An_D%C6%B0%C6%A1...
308,Thành phố Hải Phòng,Quận Dương Kinh,https://vi.wikipedia.org/wiki/D%C6%B0%C6%A1ng_...
315,Thành phố Hải Phòng,Quận Hải An,https://vi.wikipedia.org/wiki/H%E1%BA%A3i_An_(...
322,Thành phố Hải Phòng,Quận Hồng Bàng,https://vi.wikipedia.org/wiki/H%E1%BB%93ng_B%C...
333,Thành phố Hải Phòng,Quận Kiến An,https://vi.wikipedia.org/wiki/Ki%E1%BA%BFn_An_...
339,Thành phố Hải Phòng,Quận Lê Chân,https://vi.wikipedia.org/wiki/L%C3%AA_Ch%C3%A2...
347,Thành phố Hải Phòng,Quận Ngô Quyền,https://vi.wikipedia.org/wiki/Ng%C3%B4_Quy%E1%...


In [116]:
import time
import webbrowser
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

In [110]:
for _, row in not_sure_district.iterrows():
    if row['wikipedia'] == 'NO':
        pass
    else:
        webbrowser.open(row['wikipedia'])
        time.sleep(1)
        element = input('Element:')
        not_sure_district.loc[not_sure_district['wikipedia']==row['wikipedia'], 'element'] = element

In [113]:
def parse_ul(element):
    soup = BeautifulSoup(element, 'html.parser')
    lis = soup.findAll('li')
    streets = [li.text for li in lis]
    streets = [s.split(' (')[0] for s in streets]
    return ', '.join(streets)

In [120]:
def parse_table(element):
    soup = BeautifulSoup(element, 'html.parser')
    # Lấy tất cả text, phân tách theo dòng
    texts = list(soup.stripped_strings)
    texts = list(set(texts))

    # Ghép lại thành chuỗi, phân tách bằng dấu phẩy
    result = ', '.join(texts)
    return result

In [132]:
def parse_collapsible_table(element):
    soup = BeautifulSoup(element, 'html.parser')
    trs = soup.findAll('tr')
    streets = []
    for tr in trs:
        td = tr.find('td')
        if td is not None:
            steet = td.text.strip()
            streets.append(steet)
    return ', '.join(streets)

In [111]:
not_sure_district

,old_province,old_district,wikipedia,element
43,Thành phố Cần Thơ,Quận Bình Thuỷ,https://vi.wikipedia.org/wiki/B%C3%ACnh_Th%E1%...,<ul><li>Bùi Hữu Nghĩa</li>\n<li>Cách Mạng Thán...
67,Thành phố Cần Thơ,Quận Thốt Nốt,https://vi.wikipedia.org/wiki/Th%E1%BB%91t_N%E...,<ul><li>30 tháng 4</li>\n<li>Bạch Đằng</li>\n<...
214,Thành phố Hải Phòng,Huyện An Lão,NO,NaN
299,Thành phố Hải Phòng,Quận An Dương,https://vi.wikipedia.org/wiki/An_D%C6%B0%C6%A1...,<ul><li>Cống Mỹ</li>\n<li>Đống Hương</li>\n<li...
308,Thành phố Hải Phòng,Quận Dương Kinh,https://vi.wikipedia.org/wiki/D%C6%B0%C6%A1ng_...,<ul><li>An Lập</li>\n<li>An Toàn</li>\n<li>Anh...
315,Thành phố Hải Phòng,Quận Hải An,https://vi.wikipedia.org/wiki/H%E1%BA%A3i_An_(...,<ul><li>7 tháng 3</li>\n<li>An Khê</li>\n<li>A...
322,Thành phố Hải Phòng,Quận Hồng Bàng,https://vi.wikipedia.org/wiki/H%E1%BB%93ng_B%C...,<ul><li>An Chân</li>\n<li>An Lạc</li>\n<li>An ...
333,Thành phố Hải Phòng,Quận Kiến An,https://vi.wikipedia.org/wiki/Ki%E1%BA%BFn_An_...,<ul><li>An Viên</li>\n<li>Bắc Sơn</li>\n<li>Bạ...
339,Thành phố Hải Phòng,Quận Lê Chân,https://vi.wikipedia.org/wiki/L%C3%AA_Ch%C3%A2...,<ul><li>An Dương</li>\n<li>An Kim Hải</li>\n<l...
347,Thành phố Hải Phòng,Quận Ngô Quyền,https://vi.wikipedia.org/wiki/Ng%C3%B4_Quy%E1%...,<ul><li>An Đà</li>\n<li>An Khê</li>\n<li>An Ki...


In [134]:
for _, row in not_sure_district.iterrows():
    if row['wikipedia'] == 'NO':
        pass
    elif not row['element'].startswith('<'):
        not_sure_district.loc[not_sure_district['element']==row['element'], 'streets'] = row['element']
    elif row['element'].startswith('<ul'):
        streets = parse_ul(row['element'])
        not_sure_district.loc[not_sure_district['element']==row['element'], 'streets'] = streets
    elif row['element'].startswith('<table class="wikitable collapsible'):
        streets = parse_collapsible_table(row['element'])
        not_sure_district.loc[not_sure_district['element']==row['element'], 'streets'] = streets
    elif row['element'].startswith('<table'):
        streets = parse_table(row['element'])
        not_sure_district.loc[not_sure_district['element']==row['element'], 'streets'] = streets

In [135]:
not_sure_district

,old_province,old_district,wikipedia,element,streets
43,Thành phố Cần Thơ,Quận Bình Thuỷ,https://vi.wikipedia.org/wiki/B%C3%ACnh_Th%E1%...,<ul><li>Bùi Hữu Nghĩa</li>\n<li>Cách Mạng Thán...,"Bùi Hữu Nghĩa, Cách Mạng Tháng Tám, Đặng Thị N..."
67,Thành phố Cần Thơ,Quận Thốt Nốt,https://vi.wikipedia.org/wiki/Th%E1%BB%91t_N%E...,<ul><li>30 tháng 4</li>\n<li>Bạch Đằng</li>\n<...,"30 tháng 4, Bạch Đằng, Hòa Bình, Lê Lợi, Lê Th..."
214,Thành phố Hải Phòng,Huyện An Lão,NO,NaN,NaN
299,Thành phố Hải Phòng,Quận An Dương,https://vi.wikipedia.org/wiki/An_D%C6%B0%C6%A1...,<ul><li>Cống Mỹ</li>\n<li>Đống Hương</li>\n<li...,"Cống Mỹ, Đống Hương, Hà Nội, Hùng Vương, Nguyễ..."
308,Thành phố Hải Phòng,Quận Dương Kinh,https://vi.wikipedia.org/wiki/D%C6%B0%C6%A1ng_...,<ul><li>An Lập</li>\n<li>An Toàn</li>\n<li>Anh...,"An Lập, An Toàn, Anh Dũng, Bùi Phổ, Chân Kim, ..."
315,Thành phố Hải Phòng,Quận Hải An,https://vi.wikipedia.org/wiki/H%E1%BA%A3i_An_(...,<ul><li>7 tháng 3</li>\n<li>An Khê</li>\n<li>A...,"7 tháng 3, An Khê, An Kim Hải, An Trung, Bắc T..."
322,Thành phố Hải Phòng,Quận Hồng Bàng,https://vi.wikipedia.org/wiki/H%E1%BB%93ng_B%C...,<ul><li>An Chân</li>\n<li>An Lạc</li>\n<li>An ...,"An Chân, An Lạc, An Trực, Bạch Đằng, Bãi Sậy, ..."
333,Thành phố Hải Phòng,Quận Kiến An,https://vi.wikipedia.org/wiki/Ki%E1%BA%BFn_An_...,<ul><li>An Viên</li>\n<li>Bắc Sơn</li>\n<li>Bạ...,"An Viên, Bắc Sơn, Bạch Mã, Bùi Mộng Hoa, Bùi V..."
339,Thành phố Hải Phòng,Quận Lê Chân,https://vi.wikipedia.org/wiki/L%C3%AA_Ch%C3%A2...,<ul><li>An Dương</li>\n<li>An Kim Hải</li>\n<l...,"An Dương, An Kim Hải, Bùi Viện, Cát Cụt, Cầu C..."
347,Thành phố Hải Phòng,Quận Ngô Quyền,https://vi.wikipedia.org/wiki/Ng%C3%B4_Quy%E1%...,<ul><li>An Đà</li>\n<li>An Khê</li>\n<li>An Ki...,"An Đà, An Khê, An Kim Hải, Bạch Thái Bưởi, Bùi..."


In [136]:
not_sure_district.to_csv('data/address-converter.io.vn_not_sure_district', index=False)

In [142]:
not_sure_district_street = not_sure_district[~not_sure_district['streets'].isna()]
not_sure_district_street['streets'] = not_sure_district_street['streets'].str.split(',\s*')
not_sure_district_street = not_sure_district_street.explode('streets').reset_index(drop=True)
not_sure_district_street.rename(columns={'streets': 'street'}, inplace=True)

In [144]:
not_sure_district_street.to_csv('data/address-converter.io.vn_not_sure_district_street', index=False)